In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(1)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x) 

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 4

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
def g(x):  
    x_temp = torch.cos(pi*x)
    g_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return g_x

In [10]:
# Deep Ritz Method
def DRM(x):
    u_hat = model(x)
    ux = torch.zeros(x.size()[0], dimension).cuda()
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        ux[:, i] = (model(x+step_size*dx) - model(x-step_size*dx))[:,0]/step_size/2
    uxsq = (torch.sum(ux**2, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum(0.5 * uxsq + 0.5*pi**2*u_hat**2 - f_temp*u_hat)/x.size()[0]
   
    Nb = 100
    xa1 = torch.rand(Nb, dimension).cuda() 
    xa1[:, 0] = torch.zeros(Nb)
    xa2 = torch.rand(Nb, dimension).cuda() 
    xa2[:, 0] = torch.ones(Nb)
    xb1 = torch.rand(Nb, dimension).cuda()
    xb1[:, 1] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension).cuda()
    xb2[:, 1] = torch.ones(Nb)
    xc1 = torch.rand(Nb, dimension).cuda()
    xc1[:, 2] = torch.zeros(Nb)
    xc2 = torch.rand(Nb, dimension).cuda()
    xc2[:, 2] = torch.ones(Nb)
    xd1 = torch.rand(Nb, dimension).cuda()
    xd1[:, 3] = torch.zeros(Nb)
    xd2 = torch.rand(Nb, dimension).cuda()
    xd2[:, 3] = torch.ones(Nb)
    dx1 = torch.zeros(xb1.size()[0], dimension).cuda()
    dx1[:, 0] = torch.ones(xb1.size()[0])
    dx2 = torch.zeros(xb1.size()[0], dimension).cuda()
    dx2[:, 1] = torch.ones(xb1.size()[0])
    dx3 = torch.zeros(xb1.size()[0], dimension).cuda()
    dx3[:, 2] = torch.ones(xb1.size()[0])
    dx4 = torch.zeros(xb1.size()[0], dimension).cuda()
    dx4[:, 3] = torch.ones(xb1.size()[0])
    uxa1 = (((model(xa1+step_size*dx1) - model(xa1-step_size*dx1))/step_size/2 * (1.0) + model(xa1) - g(xa1))**2).cuda()
    uxa2 = (((model(xa2+step_size*dx1) - model(xa2-step_size*dx1))/step_size/2 * (-1.0) + model(xa2) - g(xa2))**2).cuda()
    uxb1 = (((model(xb1+step_size*dx2) - model(xb1-step_size*dx2))/step_size/2 * (1.0) + model(xb1) - g(xb1))**2).cuda()
    uxb2 = (((model(xb2+step_size*dx2) - model(xb2-step_size*dx2))/step_size/2 * (-1.0) + model(xb2) - g(xb2))**2).cuda()
    uxc1 = (((model(xc1+step_size*dx3) - model(xc1-step_size*dx3))/step_size/2 * (1.0) + model(xc1) - g(xc1))**2).cuda()
    uxc2 = (((model(xc2+step_size*dx3) - model(xc2-step_size*dx3))/step_size/2 * (-1.0) + model(xc2) - g(xc2))**2).cuda()
    uxd1 = (((model(xd1+step_size*dx4) - model(xd1-step_size*dx4))/step_size/2 * (1.0) + model(xd1) - g(xd1))**2).cuda()
    uxd2 = (((model(xd2+step_size*dx4) - model(xd2-step_size*dx4))/step_size/2 * (-1.0) + model(xd2) - g(xd2))**2).cuda()
    
    part_2 = torch.sum((uxa1 + uxa2 + uxb1 + uxb2 + uxc1 + uxc2 + uxd1 + uxd2)[:,0])/xa1.size()[0]
    lambda1 = 500
    return part_1 + lambda1 * part_2 / 8

In [11]:
Data_size = 500
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x.cuda()

In [12]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [13]:
model = DeepRitzNet(dimension, 8)
model = initparam(model, 0.5)

In [14]:
device = torch.device("cuda:1" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=4, out_features=8, bias=True)
  (linear1): Linear(in_features=8, out_features=8, bias=True)
  (linear2): Linear(in_features=8, out_features=8, bias=True)
  (linear3): Linear(in_features=8, out_features=8, bias=True)
  (linear4): Linear(in_features=8, out_features=8, bias=True)
  (linear5): Linear(in_features=8, out_features=8, bias=True)
  (linear6): Linear(in_features=8, out_features=8, bias=True)
  (linear_out): Linear(in_features=8, out_features=1, bias=True)
)

In [15]:
import torch.optim as optim
import torch.nn as nn
import time

In [16]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [17]:
traintime = 20000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [18]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_4D_Robin_batch_size_500.npy", error_save)
np.save("DRM_relative_error_4D_Robin_batch_size_500.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(1270.3142, device='cuda:1')
current relative error is:  tensor(0.9686, device='cuda:1')
current epoch is:  50
current loss is:  tensor(1037.5390, device='cuda:1')
current relative error is:  tensor(0.9438, device='cuda:1')
current epoch is:  100
current loss is:  tensor(443.1718, device='cuda:1')
current relative error is:  tensor(0.9047, device='cuda:1')
current epoch is:  150
current loss is:  tensor(227.9606, device='cuda:1')
current relative error is:  tensor(0.8786, device='cuda:1')
current epoch is:  200
current loss is:  tensor(127.8805, device='cuda:1')
current relative error is:  tensor(0.9586, device='cuda:1')
current epoch is:  250
current loss is:  tensor(102.6131, device='cuda:1')
current relative error is:  tensor(0.9380, device='cuda:1')
current epoch is:  300
current loss is:  tensor(71.8976, device='cuda:1')
current relative error is:  tensor(0.8626, device='cuda:1')
current epoch is:  350
current loss is:  tensor(50.5993, 

current epoch is:  3050
current loss is:  tensor(-13.2084, device='cuda:1')
current relative error is:  tensor(0.3644, device='cuda:1')
current epoch is:  3100
current loss is:  tensor(-15.7727, device='cuda:1')
current relative error is:  tensor(0.3536, device='cuda:1')
current epoch is:  3150
current loss is:  tensor(-14.8038, device='cuda:1')
current relative error is:  tensor(0.3590, device='cuda:1')
current epoch is:  3200
current loss is:  tensor(-14.6154, device='cuda:1')
current relative error is:  tensor(0.3412, device='cuda:1')
current epoch is:  3250
current loss is:  tensor(-13.5161, device='cuda:1')
current relative error is:  tensor(0.3389, device='cuda:1')
current epoch is:  3300
current loss is:  tensor(-13.8401, device='cuda:1')
current relative error is:  tensor(0.3340, device='cuda:1')
current epoch is:  3350
current loss is:  tensor(-15.8418, device='cuda:1')
current relative error is:  tensor(0.3439, device='cuda:1')
current epoch is:  3400
current loss is:  tensor

current epoch is:  6100
current loss is:  tensor(-15.6562, device='cuda:1')
current relative error is:  tensor(0.1465, device='cuda:1')
current epoch is:  6150
current loss is:  tensor(-16.9486, device='cuda:1')
current relative error is:  tensor(0.1472, device='cuda:1')
current epoch is:  6200
current loss is:  tensor(-18.6535, device='cuda:1')
current relative error is:  tensor(0.1366, device='cuda:1')
current epoch is:  6250
current loss is:  tensor(-16.8338, device='cuda:1')
current relative error is:  tensor(0.1428, device='cuda:1')
current epoch is:  6300
current loss is:  tensor(-20.0155, device='cuda:1')
current relative error is:  tensor(0.1309, device='cuda:1')
current epoch is:  6350
current loss is:  tensor(-18.3848, device='cuda:1')
current relative error is:  tensor(0.1367, device='cuda:1')
current epoch is:  6400
current loss is:  tensor(-18.0884, device='cuda:1')
current relative error is:  tensor(0.1326, device='cuda:1')
current epoch is:  6450
current loss is:  tensor

current epoch is:  9150
current loss is:  tensor(-17.8515, device='cuda:1')
current relative error is:  tensor(0.0583, device='cuda:1')
current epoch is:  9200
current loss is:  tensor(-20.4661, device='cuda:1')
current relative error is:  tensor(0.0558, device='cuda:1')
current epoch is:  9250
current loss is:  tensor(-21.1861, device='cuda:1')
current relative error is:  tensor(0.0565, device='cuda:1')
current epoch is:  9300
current loss is:  tensor(-21.0674, device='cuda:1')
current relative error is:  tensor(0.0607, device='cuda:1')
current epoch is:  9350
current loss is:  tensor(-18.6436, device='cuda:1')
current relative error is:  tensor(0.0571, device='cuda:1')
current epoch is:  9400
current loss is:  tensor(-19.5888, device='cuda:1')
current relative error is:  tensor(0.0566, device='cuda:1')
current epoch is:  9450
current loss is:  tensor(-19.3804, device='cuda:1')
current relative error is:  tensor(0.0589, device='cuda:1')
current epoch is:  9500
current loss is:  tensor

current epoch is:  12150
current loss is:  tensor(-17.6787, device='cuda:1')
current relative error is:  tensor(0.0382, device='cuda:1')
current epoch is:  12200
current loss is:  tensor(-18.4064, device='cuda:1')
current relative error is:  tensor(0.0398, device='cuda:1')
current epoch is:  12250
current loss is:  tensor(-21.1956, device='cuda:1')
current relative error is:  tensor(0.0412, device='cuda:1')
current epoch is:  12300
current loss is:  tensor(-23.0181, device='cuda:1')
current relative error is:  tensor(0.0390, device='cuda:1')
current epoch is:  12350
current loss is:  tensor(-20.1489, device='cuda:1')
current relative error is:  tensor(0.0389, device='cuda:1')
current epoch is:  12400
current loss is:  tensor(-17.8083, device='cuda:1')
current relative error is:  tensor(0.0355, device='cuda:1')
current epoch is:  12450
current loss is:  tensor(-16.6849, device='cuda:1')
current relative error is:  tensor(0.0385, device='cuda:1')
current epoch is:  12500
current loss is:

current epoch is:  15150
current loss is:  tensor(-17.8505, device='cuda:1')
current relative error is:  tensor(0.0278, device='cuda:1')
current epoch is:  15200
current loss is:  tensor(-18.8466, device='cuda:1')
current relative error is:  tensor(0.0311, device='cuda:1')
current epoch is:  15250
current loss is:  tensor(-19.0984, device='cuda:1')
current relative error is:  tensor(0.0304, device='cuda:1')
current epoch is:  15300
current loss is:  tensor(-16.1045, device='cuda:1')
current relative error is:  tensor(0.0276, device='cuda:1')
current epoch is:  15350
current loss is:  tensor(-20.9832, device='cuda:1')
current relative error is:  tensor(0.0287, device='cuda:1')
current epoch is:  15400
current loss is:  tensor(-19.4022, device='cuda:1')
current relative error is:  tensor(0.0306, device='cuda:1')
current epoch is:  15450
current loss is:  tensor(-21.5644, device='cuda:1')
current relative error is:  tensor(0.0285, device='cuda:1')
current epoch is:  15500
current loss is:

current epoch is:  18150
current loss is:  tensor(-22.4129, device='cuda:1')
current relative error is:  tensor(0.0274, device='cuda:1')
current epoch is:  18200
current loss is:  tensor(-20.9576, device='cuda:1')
current relative error is:  tensor(0.0242, device='cuda:1')
current epoch is:  18250
current loss is:  tensor(-19.3614, device='cuda:1')
current relative error is:  tensor(0.0223, device='cuda:1')
current epoch is:  18300
current loss is:  tensor(-18.2997, device='cuda:1')
current relative error is:  tensor(0.0267, device='cuda:1')
current epoch is:  18350
current loss is:  tensor(-18.8011, device='cuda:1')
current relative error is:  tensor(0.0232, device='cuda:1')
current epoch is:  18400
current loss is:  tensor(-19.5621, device='cuda:1')
current relative error is:  tensor(0.0238, device='cuda:1')
current epoch is:  18450
current loss is:  tensor(-17.8810, device='cuda:1')
current relative error is:  tensor(0.0256, device='cuda:1')
current epoch is:  18500
current loss is: